In [1]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   
from pyntcloud import PyntCloud

#Loading the point cloud

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedCloud.ply")) 




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Making a voxelgrid from the point cloud
voxel_size = 3
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [3]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [ ]:
#This is old code, not if needed anymore
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

#Extracting points from each voxel, calculating features for all of them and storing them in lists
voxel_size = 3


linearity =[]
planarity = []
scattering = []
omnivariance = []
sum_eigenvalues = []
anisotropy = []
eigentropy = []
change_of_curvature = []
points_per_voxel = []
z_range = []
 

#Calculating Z-range and standard deviation from the plane for each voxel

#Z-range

#Defining a function to calculate the z range
def calculateZRange(points):
    if len(points) < 15:
        return 0
    return(np.max(points[:,2]) - np.min(points[:,2]))
   
#Run this to just run it on the first 100 voxels
#voxel_center_coordinates = voxel_center_coordinates[:100]


for center in tqdm(voxel_center_coordinates):
    minboundx = center[0] - voxel_size/2
    minboundy = center[1] - voxel_size/2
    minboundz = center[2] - voxel_size/2

    maxboundx = center[0] + voxel_size/2
    maxboundy = center[1] + voxel_size/2
    maxboundz = center[2] + voxel_size/2

    voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
    #print("Number of points in voxel ", len(voxel_extracted_points.points))
    
    points = np.asarray(voxel_extracted_points.points)
    
    #Checking if there are enough points in the voxel to calculate PCA
    
    if len(points) < 15:
        linearity.append(0)
        planarity.append(0)
        scattering.append(0)
        omnivariance.append(0)
        sum_eigenvalues.append(0)
        anisotropy.append(0)
        eigentropy.append(0)
        change_of_curvature.append(0)
        points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
        continue
    
    pca = PCA(n_components=3)
    pca.fit(points)

    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    
   
    

In [4]:
#Defining a function to extract points from a voxel based on center coordinates. Returns a list of points

from sklearn.decomposition import PCA
from tqdm.auto import tqdm


def extractPointCloudFromVoxel(voxel_center_coordinates, pointCloud, voxel_size):
    extractedPointClouds = []
    for center in tqdm(voxel_center_coordinates):
        minboundx = center[0] - voxel_size/2
        minboundy = center[1] - voxel_size/2
        minboundz = center[2] - voxel_size/2

        maxboundx = center[0] + voxel_size/2
        maxboundy = center[1] + voxel_size/2
        maxboundz = center[2] + voxel_size/2

        pcd = pointCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
        #print("Number of points in voxel ", len(voxel_extracted_points.points))
        extractedPointClouds.append(pcd)
    return extractedPointClouds
    
    
    

In [5]:
#Function to calculate features from a list of points clouds (could be one or more), returns lists for each feature
def calculatePCAFeatures(listOfPointClouds):
    
    linearity =[]
    planarity = []
    scattering = []
    omnivariance = []
    sum_eigenvalues = []
    anisotropy = []
    eigentropy = []
    change_of_curvature = []
    eigenvaluesList = []
    eigenvectorsList = []
    points_per_voxel = []
    
    for pcd in tqdm(listOfPointClouds):
        points = pcd.points
    
        if len(points) < 15:
            linearity.append(0)
            planarity.append(0)
            scattering.append(0)
            omnivariance.append(0)
            sum_eigenvalues.append(0)
            anisotropy.append(0)
            eigentropy.append(0)
            change_of_curvature.append(0)
            points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
            continue
    
    
        pca = PCA(n_components=3)
        pca.fit(points)

    # Extract the eigenvalues and eigenvectors
        eigenvalues = pca.explained_variance_
        eigenvectors = pca.components_
    
      #Calculating the features
        l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
        p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
        s = eigenvalues[2] / eigenvalues[0]
        o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
        s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
        a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
        e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
        c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])

        linearity.append(l)
        planarity.append(p)
        scattering.append(s)
        omnivariance.append(o)
        sum_eigenvalues.append(s_e)
        anisotropy.append(a)
        eigentropy.append(e)
        change_of_curvature.append(c_o_c)
        eigenvaluesList.append(eigenvalues)
        eigenvectorsList.append(eigenvectors)
        points_per_voxel.append(points)
        
    return linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvaluesList, eigenvectorsList, points_per_voxel

In [6]:
#Defining a seperate function to calculate z-range for a set of points
def calculateZRange(points):
    z_range = []
    if len(points) < 15:
        z_range.append(0)
        z = np.max(points[:,2]) - np.min(points[:,2])
        z_range.append(z)
    return z_range
   

In [7]:
#Extracting the points from each voxel

extractedPointClouds = extractPointCloudFromVoxel(voxel_center_coordinates, initialCloud, voxel_size)

  0%|          | 0/29528 [00:00<?, ?it/s]

In [8]:
#Calculating the features for each voxel
linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvalues, eigenvectors, points_per_voxel = calculatePCAFeatures(extractedPointClouds)

  0%|          | 0/29528 [00:00<?, ?it/s]

In [ ]:
#Calculating z-range for each voxel (var noe galt her, må finne ut senere)

z_range = []
for pcd in tqdm(extractedPointClouds):
    points = np.asarray(pcd.points)
    if len(points) < 15:
        z_range.append(0)
        continue
    z = calculateZRange(points)
    z_range.append(z)


In [9]:
#Making a dataframe with all the features and the points for each voxel with an empty label column

import pandas as pd

# both lists, with columns specified
label = [0] * len(linearity) #Placeholder zeroes

df = pd.DataFrame(list(zip(points_per_voxel, linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, label)),
			columns =[ 'points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'label'])



In [10]:
df

,points,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,label
0,"[[570149.970001, 7034352.429993, 4.96], [57015...",0.439892,0.085995,0.474113,0.581093,1.839046,0.525887,0.798887,0.233069,0
1,"[[570268.780029, 7034220.920013, 27.32], [5702...",0.429511,0.557233,0.013256,0.167289,1.349800,0.986744,0.537485,0.008370,0
2,"[[570331.419983, 7034335.01001, 20.65], [57033...",0.164252,0.835163,0.000585,0.064187,1.496035,0.999415,0.432333,0.000319,0
3,"[[570203.660004, 7034216.220001, 24.42], [5702...",0.705547,0.261180,0.033273,0.208933,1.296424,0.966727,0.493010,0.025060,0
4,"[[570257.23999, 7034343.890015, 16.940001], [5...",0.501917,0.370362,0.127722,0.107745,0.438810,0.872278,0.739346,0.078559,0
...,...,...,...,...,...,...,...,...,...,...
29523,"[[570216.380005, 7034328.049988, 18.780001]]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
29524,"[[570223.340027, 7034311.540009, 12.65], [5702...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
29525,"[[570203.959991, 7034358.919983, 6.68]]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
29526,"[[570234.849976, 7034343.909973, 17.35], [5702...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [44]:
import os
#Read all the houses into a list

houses_folder = os.path.join(os.getcwd(), "data", "Houses")
houses_files = os.listdir(houses_folder)

houses = []
for file in houses_files:
    if file.endswith(".ply"):
        house = o3d.io.read_point_cloud(os.path.join(houses_folder, file))
        houses.append(house)


In [45]:
#Load all the trees into a list

trees_folder = os.path.join(os.getcwd(), "data", "Trees")   
trees_files = os.listdir(trees_folder)

trees = []
for file in trees_files:
    if file.endswith(".ply"):
        tree = o3d.io.read_point_cloud(os.path.join(trees_folder, file))
        trees.append(tree)


In [46]:
#Load all terrain into a list

terrain_folder = os.path.join(os.getcwd(), "data", "Terrain")
terrain_files = os.listdir(terrain_folder)

terrain = []
for file in terrain_files:
    if file.endswith(".ply"):
        terrain_piece = o3d.io.read_point_cloud(os.path.join(terrain_folder, file))
        terrain.append(terrain_piece)


In [84]:
#Parameters for houses

#First making a point cloud of all houses
houseCloud = o3d.geometry.PointCloud()
for house in houses:
    houseCloud+= house
    
#Making a voxel grid of the houses
houseVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(houseCloud, voxel_size)

#Extracting center points
houseVoxelCenterCoordinates= []
for i in houseVoxelGrid.get_voxels():
    houseVoxelCenterCoordinates.append(houseVoxelGrid.get_voxel_center_coordinate(i.grid_index))
    
#Extracting points from each voxel
houseExtractedPointClouds = extractPointCloudFromVoxel(houseVoxelCenterCoordinates, houseCloud, voxel_size)

#Calculating the features for each voxel
houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseEigenvalues, houseEigenvectors, housePoints_per_voxel = calculatePCAFeatures(houseExtractedPointClouds)

#Making an array of labels for the houses

houseLabels = np.ones(len(houseLinearity))

#Making a dataframe with all the features and the points for each voxel
dfHouse = pd.DataFrame(list(zip(housePoints_per_voxel, houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'label'])

  0%|          | 0/2991 [00:00<?, ?it/s]

  0%|          | 0/2991 [00:00<?, ?it/s]

In [85]:
#Parameters for trees

#First making a point cloud of all trees
treeCloud = o3d.geometry.PointCloud()
for tree in trees:
    treeCloud+= tree
    
#Making a voxel grid of the trees
treeVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(treeCloud, voxel_size)

#Extracting center points
treeVoxelCenterCoordinates= []
for i in treeVoxelGrid.get_voxels():
    treeVoxelCenterCoordinates.append(treeVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
treeExtractedPointClouds = extractPointCloudFromVoxel(treeVoxelCenterCoordinates, treeCloud, voxel_size)

#Calculating the features for each voxel
treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeEigenvalues, treeEigenvectors, treePoints_per_voxel = calculatePCAFeatures(treeExtractedPointClouds)

#Making an array of labels for the trees
treeLabels = np.zeros(len(treeLinearity))   

#Making a dataframe with all the features and the points for each voxel
dfTree = pd.DataFrame(list(zip(treePoints_per_voxel, treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'label'])



  0%|          | 0/4342 [00:00<?, ?it/s]

  0%|          | 0/4342 [00:00<?, ?it/s]

In [86]:
#Parameters for terrain

#First making a point cloud of all terrain
terrainCloud = o3d.geometry.PointCloud()

for terrain_piece in terrain:
    terrainCloud+= terrain_piece

#Making a voxel grid of the terrain
terrainVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(terrainCloud, voxel_size)

#Extracting center points
terrainVoxelCenterCoordinates= []
for i in terrainVoxelGrid.get_voxels():
    terrainVoxelCenterCoordinates.append(terrainVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
terrainExtractedPointClouds = extractPointCloudFromVoxel(terrainVoxelCenterCoordinates, terrainCloud, voxel_size)

#Calculating the features for each voxel
terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainEigenvalues, terrainEigenvectors, terrainPoints_per_voxel = calculatePCAFeatures(terrainExtractedPointClouds)

#Making an array of labels for the terrain with number 2
terrainLabels = np.full(len(terrainLinearity), 2)

#Making a dataframe with all the features and the points for each voxel
dfTerrain = pd.DataFrame(list(zip(terrainPoints_per_voxel, terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'label'])




  0%|          | 0/1971 [00:00<?, ?it/s]

  0%|          | 0/1971 [00:00<?, ?it/s]

In [109]:
#Now its time for the machine learning part

#First, we need to split the data into training and testing data, and also try to drop some features to see if it improves the accuracy

from sklearn.model_selection import train_test_split

#Combining the datasets into one dataframe
dfCombined = pd.concat([dfHouse, dfTree, dfTerrain], ignore_index=True)


#Placing the labels in a dataframe
y_train = dfCombined['label']

#Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfCombined.drop(['points', 'label', 'linearity', 'scattering', 'eigentropy'], axis=1), dfCombined['label'], test_size=0.33, random_state=42)




In [110]:
#Training a decision tree classifier

from sklearn.tree import DecisionTreeClassifier

#Creating the classifier
clf = DecisionTreeClassifier(random_state=0)

#Training the classifier
clf.fit(X_train, y_train)


#Testing it
y_pred = clf.predict(X_test)



In [111]:
#Testing the accuracy of the classifier

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

feature_importances = clf.feature_importances_
print(feature_importances)



0.6173884728101595
[0.0892263  0.10549662 0.04624865 0.09475385 0.1020784  0.27584409
 0.09750668 0.1888454 ]


In [53]:
#Now actually testing it on the point cloud

#We already have the dataframe for the entire point cloud, so we can just use that
#We need to drop the points column and the label column
X_cloud = df.drop(['points', 'label'], axis=1)

#Predicting the labels for the point cloud
y_cloud_pred = clf.predict(X_cloud)

In [54]:
#Adding the labels to the dataframe
df['label'] = y_cloud_pred



In [55]:
df

,points,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,label
0,"[[570149.970001, 7034352.429993, 4.96], [57015...",0.439892,0.085995,0.474113,0.581093,1.839046,0.525887,0.798887,0.233069,0.0
1,"[[570268.780029, 7034220.920013, 27.32], [5702...",0.429511,0.557233,0.013256,0.167289,1.349800,0.986744,0.537485,0.008370,1.0
2,"[[570331.419983, 7034335.01001, 20.65], [57033...",0.164252,0.835163,0.000585,0.064187,1.496035,0.999415,0.432333,0.000319,2.0
3,"[[570203.660004, 7034216.220001, 24.42], [5702...",0.705547,0.261180,0.033273,0.208933,1.296424,0.966727,0.493010,0.025060,0.0
4,"[[570257.23999, 7034343.890015, 16.940001], [5...",0.501917,0.370362,0.127722,0.107745,0.438810,0.872278,0.739346,0.078559,1.0
...,...,...,...,...,...,...,...,...,...,...
29523,"[[570216.380005, 7034328.049988, 18.780001]]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
29524,"[[570223.340027, 7034311.540009, 12.65], [5702...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
29525,"[[570203.959991, 7034358.919983, 6.68]]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
29526,"[[570234.849976, 7034343.909973, 17.35], [5702...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [56]:
#Now we need to extract the points from the trees and houses

#Making a dataframe for trees and houses
dfTrees = df[df['label'] == 0]
dfHouses = df[df['label'] == 1]
dfTerrain = df[df['label'] == 2]

#Extracting the point-clusters from the trees
treeClusters= dfTrees['points']
houseClusters = dfHouses['points']
terrainClusters = dfTerrain['points']

classifiedTreeCloud = o3d.geometry.PointCloud()
classifiedHouseCloud = o3d.geometry.PointCloud()
classifiedTerrainCloud = o3d.geometry.PointCloud()

#Painting them

#Trees get green
for cluster in treeClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([0, 1, 0])  
    classifiedTreeCloud += cld

#Houses get red
for cluster in houseClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([1, 0, 0])  
    classifiedHouseCloud += cld
    
#Terrain gets yellow
for cluster in terrainClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([1, 1, 0])
    classifiedTerrainCloud += cld


In [57]:
o3d.visualization.draw_geometries([classifiedHouseCloud, classifiedTreeCloud, classifiedTerrainCloud, initialCloud])